# Dataset

CIFAR 10

In [1]:
import torchvision

cifar10Train = torchvision.datasets.CIFAR10("./CIFAR10", download=True, transform=lambda im: torchvision.transforms.functional.pil_to_tensor(im)/255)
cifar10Test = torchvision.datasets.CIFAR10("./CIFAR10", train=False, download=True, transform=lambda im: torchvision.transforms.functional.pil_to_tensor(im)/255)

Files already downloaded and verified
Files already downloaded and verified


In [2]:
import numpy as np
import pandas as pd
import time

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import PIL
from sklearn.metrics import accuracy_score, f1_score

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Models

## CNN1.2

3 Conv, 2 FC, No dropout, no batch norm

Activation fn: ReLU

Optimizer: Adam

In [3]:
class CNN1(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn1 = nn.Conv2d(3, 16, (3, 3), padding='same').to(device)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d((2, 2), stride=(2, 2)).to(device)

        self.cnn2 = nn.Conv2d(16, 32, (3, 3), padding='same').to(device)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d((2, 2), stride=(2,2)).to(device)

        self.cnn3 = nn.Conv2d(32, 64, (3, 3), padding='same').to(device)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d((2, 2), stride=(2,2)).to(device)

        self.linear1 = nn.Linear(64 * 4 * 4, 512)
        self.relu4 = nn.ReLU()

        self.linear2 = nn.Linear(512, 512)
        self.relu5 = nn.ReLU()

        self.linear3 = nn.Linear(512, 10)

    def forward(self, inputs):
        x = self.cnn1(inputs)
        x = self.relu1(x)
        x = self.maxpool1(x)

        x = self.cnn2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)

        x = self.cnn3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)

        x = torch.flatten(x, 1)

        x = self.linear1(x)
        x = self.relu4(x)

        x = self.linear2(x)
        x = self.relu5(x)

        x = self.linear3(x)
    
        return x


In [4]:
trainloader = DataLoader(cifar10Train, 64)
testloader = DataLoader(cifar10Test, 64)

In [5]:
def train_one_epoch(model, optimzer, loss_fn, training_loader):
    running_loss = 0.
    running_momentum = 0.

    for i, data in enumerate(training_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs.to(device))

        loss = loss_fn(outputs, labels.to(device))
        loss.backward()

        optimizer.step()
           

        running_loss += loss.item()

        
    average_loss = running_loss/(i+1)
    

    return average_loss

In [6]:
def evaluate(tdataset, model):
    model.eval()
    preds = []
    truths = []
    for example in tdataset:
        input = example[0].unsqueeze(0).to(device)
        logits = model(input)
        pred = torch.argmax(torch.softmax(logits, 1))
        preds.append(pred.item())
        truths.append(example[1])
    return accuracy_score(truths, preds), f1_score(truths, preds, average='macro'), preds, truths

In [7]:
def save_model(model, filepath):
    torch.save(model.state_dict(), filepath)

def load_model(model, filepath, device='cpu'):
    model.load_state_dict(torch.load(filepath, map_location=device))
    model.eval()

In [8]:
cnn1 = CNN1().to(device)

optimizer = torch.optim.Adam(cnn1.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

/home/aparna/whisper_transcription/env/lib/python3.8/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [9]:
EPOCHS = 20

begin = time.time()

max_f1 = -float('inf')

log = []

for epoch in range(EPOCHS):
    cnn1.train(True)
    average_train_loss = train_one_epoch(cnn1, optimizer, loss_fn, trainloader)

    cnn1.eval()
    running_test_loss = 0.
    for i, data in enumerate(testloader):
        inputs, labels = data
        outputs = cnn1(inputs.to(device))
        loss = loss_fn(outputs, labels.to(device))
        running_test_loss += loss.item()
    average_test_loss = running_test_loss/(i+1)
    end = time.time()
    log.append({'average_test_loss': average_test_loss, 'average_train_loss': average_train_loss, 'time_from_start': end-begin})
    if (epoch+1)%1 == 0:
        train_accuracy, train_f1, _, _ = evaluate(cifar10Train, cnn1)
        test_accuracy, test_f1, _, _ = evaluate(cifar10Test, cnn1)
        if test_f1 > max_f1:
            max_f1 = test_f1
            save_model(cnn1, './models/cnn1-2.pth')
        print(f"Epoch {epoch+1} | train loss: {average_train_loss:.3f} | train accuracy: {100*train_accuracy:.2f}% | train f1: {train_f1:.2f} | test loss: {average_test_loss:.3f} | test accuracy: {100*test_accuracy:.2f}% | test f1: {test_f1:.2f} | time: {end-begin:.2f}s")

Epoch 1 | train loss: 1.603 | train accuracy: 53.24% | train f1: 0.52 | test loss: 1.302 | test accuracy: 52.44% | test f1: 0.52 | time: 11.85s
Epoch 2 | train loss: 1.177 | train accuracy: 63.91% | train f1: 0.64 | test loss: 1.068 | test accuracy: 61.68% | test f1: 0.61 | time: 81.72s
Epoch 3 | train loss: 0.985 | train accuracy: 68.35% | train f1: 0.69 | test loss: 1.014 | test accuracy: 64.65% | test f1: 0.65 | time: 151.55s
Epoch 4 | train loss: 0.855 | train accuracy: 71.02% | train f1: 0.72 | test loss: 1.019 | test accuracy: 65.40% | test f1: 0.66 | time: 221.47s
Epoch 5 | train loss: 0.750 | train accuracy: 75.60% | train f1: 0.76 | test loss: 0.955 | test accuracy: 67.89% | test f1: 0.68 | time: 291.82s
Epoch 6 | train loss: 0.658 | train accuracy: 77.62% | train f1: 0.78 | test loss: 0.994 | test accuracy: 67.99% | test f1: 0.69 | time: 362.33s
Epoch 7 | train loss: 0.588 | train accuracy: 79.66% | train f1: 0.80 | test loss: 1.054 | test accuracy: 68.17% | test f1: 0.69 | t

In [10]:
import pickle

pickle.dump(log, open('./logs/cnn1-2.pkl', 'wb'))